# [실습] LangChain을 이용한 SQL 데이터베이스 분석

LangChain은 다양한 데이터 형식과 연동됩니다.    
이번 실습에서는 SQL 데이터베이스와 연동하기 위해, SQL 쿼리를 생성하고 실제로 실행하는 방법을 활용해 보겠습니다.

In [ ]:
!pip install openai langchain langchain-community langchain-openai matplotlib dotenv

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32

In [ ]:
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv('env',override=True)


llm = ChatOpenAI(model = 'gpt-5-mini', # temperature=0.1,
                 reasoning_effort='low')

In [ ]:
llm.invoke("SQL DB의 핵심 명령어 5개 알려줘")

AIMessage(content="다음은 SQL(관계형 DB)에서 가장 핵심적인 5가지 명령어와 간단한 설명·예시입니다.\n\n1) SELECT — 데이터 조회\n- 설명: 테이블에서 행(레코드)을 조회한다.\n- 기본 문법: SELECT 컬럼 FROM 테이블 WHERE 조건;\n- 예: SELECT id, name FROM users WHERE active = 1 ORDER BY created_at DESC;\n\n2) INSERT — 데이터 삽입\n- 설명: 테이블에 새로운 행을 추가한다.\n- 기본 문법: INSERT INTO 테이블 (컬럼1, 컬럼2) VALUES (값1, 값2);\n- 예: INSERT INTO users (name, email, active) VALUES ('홍길동', 'hong@example.com', 1);\n\n3) UPDATE — 데이터 수정\n- 설명: 기존 행의 값을 변경한다(보통 WHERE로 대상 지정).\n- 기본 문법: UPDATE 테이블 SET 컬럼1 = 값1, 컬럼2 = 값2 WHERE 조건;\n- 예: UPDATE users SET active = 0 WHERE id = 10;\n\n4) DELETE — 데이터 삭제\n- 설명: 테이블에서 행을 삭제한다(주의: WHERE 없으면 전체 삭제).\n- 기본 문법: DELETE FROM 테이블 WHERE 조건;\n- 예: DELETE FROM users WHERE id = 10;\n\n5) CREATE TABLE — 테이블 생성(스키마 정의)\n- 설명: 새 테이블과 컬럼(타입, 제약조건)을 만든다.\n- 기본 문법: CREATE TABLE 테이블명 (컬럼명 타입 제약조건, ...);\n- 예: \n  CREATE TABLE users (\n    id INT PRIMARY KEY AUTO_INCREMENT,\n    name VARCHAR(100) NOT NULL,\n    email VARCHAR(255) UNIQUE,\n    created_at TIMESTAMP

## 1. 예제 SQL 파일 만들기   

가상의 도서관 데이터베이스를 만들어 활용하겠습니다.   
책, 유저, 대여 정보로 구성된 3개의 테이블을 사용하겠습니다.

In [ ]:
import sqlite3
from datetime import datetime, timedelta

# 현재 날짜 기준 설정
today = datetime.now().date()

# SQLite DB 파일 연결
conn = sqlite3.connect('library.db')
c = conn.cursor()

# 1. 출판사 테이블
c.execute('''
CREATE TABLE IF NOT EXISTS publishers (
    publisher_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    country TEXT,
    established_year INTEGER
)
''')

# 2. 저자 테이블 (다대다 관계를 위해 분리)
c.execute('''
CREATE TABLE IF NOT EXISTS authors (
    author_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    birth_year INTEGER,
    nationality TEXT
)
''')

# 3. 카테고리 테이블 (정규화)
c.execute('''
CREATE TABLE IF NOT EXISTS categories (
    category_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    description TEXT
)
''')

# 4. 책 테이블 (출판사, 카테고리 외래키 추가)
c.execute('''
CREATE TABLE IF NOT EXISTS books (
    book_id INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    isbn TEXT UNIQUE,
    publication_year INTEGER,
    publisher_id INTEGER,
    category_id INTEGER,
    page_count INTEGER,
    language TEXT DEFAULT 'Korean',
    FOREIGN KEY(publisher_id) REFERENCES publishers(publisher_id),
    FOREIGN KEY(category_id) REFERENCES categories(category_id)
)
''')

# 5. 책-저자 연결 테이블 (다대다 관계)
c.execute('''
CREATE TABLE IF NOT EXISTS book_authors (
    book_id INTEGER,
    author_id INTEGER,
    author_order INTEGER,
    PRIMARY KEY(book_id, author_id),
    FOREIGN KEY(book_id) REFERENCES books(book_id),
    FOREIGN KEY(author_id) REFERENCES authors(author_id)
)
''')

# 6. 책 복본 테이블 (실제 물리적 책)
c.execute('''
CREATE TABLE IF NOT EXISTS book_copies (
    copy_id INTEGER PRIMARY KEY,
    book_id INTEGER,
    barcode TEXT UNIQUE,
    condition TEXT CHECK(condition IN ('excellent', 'good', 'fair', 'poor')),
    location TEXT,
    acquisition_date TEXT,
    status TEXT DEFAULT 'available' CHECK(status IN ('available', 'borrowed', 'lost', 'repair')),
    FOREIGN KEY(book_id) REFERENCES books(book_id)
)
''')

# 7. 회원 등급 테이블
c.execute('''
CREATE TABLE IF NOT EXISTS membership_types (
    type_id INTEGER PRIMARY KEY,
    type_name TEXT NOT NULL,
    max_books INTEGER,
    loan_period_days INTEGER,
    late_fee_per_day REAL
)
''')

# 8. 회원 테이블 (등급 추가)
c.execute('''
CREATE TABLE IF NOT EXISTS members (
    member_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    email TEXT UNIQUE,
    phone TEXT,
    address TEXT,
    join_date TEXT,
    membership_type_id INTEGER DEFAULT 1,
    is_active INTEGER DEFAULT 1,
    FOREIGN KEY(membership_type_id) REFERENCES membership_types(type_id)
)
''')

# 9. 대여 정보 테이블 (실제 복본 기준)
c.execute('''
CREATE TABLE IF NOT EXISTS rentals (
    rental_id INTEGER PRIMARY KEY,
    copy_id INTEGER,
    member_id INTEGER,
    rental_date TEXT,
    due_date TEXT,
    return_date TEXT,
    status TEXT DEFAULT 'active' CHECK(status IN ('active', 'returned', 'overdue')),
    FOREIGN KEY(copy_id) REFERENCES book_copies(copy_id),
    FOREIGN KEY(member_id) REFERENCES members(member_id)
)
''')

# 10. 예약 테이블
c.execute('''
CREATE TABLE IF NOT EXISTS reservations (
    reservation_id INTEGER PRIMARY KEY,
    book_id INTEGER,
    member_id INTEGER,
    reservation_date TEXT,
    status TEXT DEFAULT 'waiting' CHECK(status IN ('waiting', 'ready', 'cancelled', 'completed')),
    expiry_date TEXT,
    FOREIGN KEY(book_id) REFERENCES books(book_id),
    FOREIGN KEY(member_id) REFERENCES members(member_id)
)
''')

# 11. 연체료 테이블
c.execute('''
CREATE TABLE IF NOT EXISTS late_fees (
    fee_id INTEGER PRIMARY KEY,
    rental_id INTEGER,
    amount REAL,
    calculation_date TEXT,
    paid INTEGER DEFAULT 0,
    payment_date TEXT,
    FOREIGN KEY(rental_id) REFERENCES rentals(rental_id)
)
''')

# 12. 리뷰 테이블
c.execute('''
CREATE TABLE IF NOT EXISTS reviews (
    review_id INTEGER PRIMARY KEY,
    book_id INTEGER,
    member_id INTEGER,
    rating INTEGER CHECK(rating >= 1 AND rating <= 5),
    comment TEXT,
    review_date TEXT,
    FOREIGN KEY(book_id) REFERENCES books(book_id),
    FOREIGN KEY(member_id) REFERENCES members(member_id)
)
''')

print("테이블 생성 완료!")

# === 데이터 삽입 ===

# 1. 출판사 데이터
publishers_data = [
    ('문학동네', '대한민국', 1993),
    ('민음사', '대한민국', 1966),
    ('Bloomsbury', '영국', 1986),
    ('Scribner', '미국', 1846),
    ('Penguin Books', '영국', 1935)
]
c.executemany("INSERT INTO publishers (name, country, established_year) VALUES (?, ?, ?)", publishers_data)

# 2. 카테고리 데이터
categories_data = [
    ('판타지', '마법, 신화, 초자연적 요소가 포함된 소설'),
    ('고전문학', '시대를 초월한 문학적 가치를 지닌 작품'),
    ('SF', '과학과 기술을 기반으로 한 공상과학 소설'),
    ('소설', '일반 문학 소설'),
    ('디스토피아', '암울한 미래 사회를 다룬 작품'),
    ('동화', '어린이를 위한 이야기')
]
c.executemany("INSERT INTO categories (name, description) VALUES (?, ?)", categories_data)

# 3. 저자 데이터
authors_data = [
    ('J.K. 롤링', 1965, '영국'),
    ('어니스트 헤밍웨이', 1899, '미국'),
    ('김초엽', 1993, '대한민국'),
    ('헤르만 헤세', 1877, '독일'),
    ('J.R.R. 톨킨', 1892, '영국'),
    ('조지 오웰', 1903, '영국'),
    ('루이스 캐럴', 1832, '영국'),
    ('F. 스콧 피츠제럴드', 1896, '미국'),
    ('이민진', 1968, '대한민국'),
    ('J.D. Salinger', 1919, '미국'),
    ('Aldous Huxley', 1894, '영국')
]
c.executemany("INSERT INTO authors (name, birth_year, nationality) VALUES (?, ?, ?)", authors_data)

# 4. 책 데이터
books_data = [
    ('해리 포터와 마법사의 돌', '9788983920003', 1997, 3, 1, 320, 'Korean'),
    ('노인과 바다', '9788937460012', 1952, 4, 2, 128, 'Korean'),
    ('지구 끝의 온실', '9791164371273', 2016, 1, 3, 256, 'Korean'),
    ('데미안', '9788937462788', 1919, 2, 4, 224, 'Korean'),
    ('반지의 제왕', '9788937460893', 1954, 2, 1, 576, 'Korean'),
    ('1984', '9788937462672', 1949, 5, 5, 400, 'Korean'),
    ('이상한 나라의 앨리스', '9788937460821', 1865, 5, 6, 192, 'Korean'),
    ('위대한 개츠비', '9788937462627', 1925, 4, 4, 256, 'Korean'),
    ('파친코', '9788932473536', 2017, 1, 4, 640, 'Korean'),
    ('The Catcher in the Rye', '9780316769174', 1951, 4, 4, 288, 'English'),
    ('Brave New World', '9780060850524', 1932, 5, 5, 311, 'English')
]
c.executemany("""INSERT INTO books (title, isbn, publication_year, publisher_id, category_id,
              page_count, language) VALUES (?, ?, ?, ?, ?, ?, ?)""", books_data)

# 5. 책-저자 연결
book_authors_data = [
    (1, 1, 1), (2, 2, 1), (3, 3, 1), (4, 4, 1),
    (5, 5, 1), (6, 6, 1), (7, 7, 1), (8, 8, 1),
    (9, 9, 1), (10, 10, 1), (11, 11, 1)
]
c.executemany("INSERT INTO book_authors (book_id, author_id, author_order) VALUES (?, ?, ?)", book_authors_data)

# 6. 책 복본 생성 (각 책당 2-3권, 현재 날짜 기준으로 입고일 설정)
book_copies_data = []
for book_id in range(1, 12):
    num_copies = 2 if book_id % 2 == 0 else 3
    for i in range(num_copies):
        barcode = f"BC{book_id:03d}{i+1:02d}"
        condition = 'excellent' if i == 0 else 'good'
        location = f"A{(book_id-1)//3 + 1}-{(book_id-1)%3 + 1}"
        # 입고일을 6개월 전부터 최근까지 다양하게 설정
        acquisition_days_ago = 180 - (book_id * 10) - (i * 5)
        acquisition_date = (today - timedelta(days=acquisition_days_ago)).isoformat()
        status = 'available' if i > 0 else 'borrowed'
        book_copies_data.append((book_id, barcode, condition, location, acquisition_date, status))

c.executemany("""INSERT INTO book_copies (book_id, barcode, condition, location, acquisition_date, status)
              VALUES (?, ?, ?, ?, ?, ?)""", book_copies_data)

# 7. 회원 등급
membership_types_data = [
    ('일반', 3, 14, 100),
    ('우수', 5, 21, 50),
    ('VIP', 10, 30, 0)
]
c.executemany("""INSERT INTO membership_types (type_name, max_books, loan_period_days, late_fee_per_day)
              VALUES (?, ?, ?, ?)""", membership_types_data)

# 8. 회원 데이터 (가입일을 최근 날짜로 설정)
members_data = [
    ('김민준', 'minjun@example.com', '010-1234-5678', '서울시 강남구',
     (today - timedelta(days=60)).isoformat(), 1, 1),
    ('이하늘', 'haneul@example.com', '010-2345-6789', '서울시 서초구',
     (today - timedelta(days=55)).isoformat(), 2, 1),
    ('박서준', 'seojoon@example.com', '010-3456-7890', '서울시 송파구',
     (today - timedelta(days=50)).isoformat(), 1, 1),
    ('최유리', 'yuri@example.com', '010-4567-8901', '서울시 마포구',
     (today - timedelta(days=45)).isoformat(), 3, 1),
    ('정다은', 'daeun@example.com', '010-5678-9012', '서울시 용산구',
     (today - timedelta(days=40)).isoformat(), 2, 1)
]
c.executemany("""INSERT INTO members (name, email, phone, address, join_date, membership_type_id, is_active)
              VALUES (?, ?, ?, ?, ?, ?, ?)""", members_data)

# 9. 대여 정보 (현재 날짜 기준으로 다양한 상황 생성)
rentals_data = [
    # 정상 대여중 (반납일이 미래)
    (1, 1, (today - timedelta(days=10)).isoformat(),
           (today + timedelta(days=4)).isoformat(), None, 'active'),

    # 연체중 (반납일 지남, 아직 반납 안함) - 5일 연체
    (4, 2, (today - timedelta(days=26)).isoformat(),
           (today - timedelta(days=5)).isoformat(), None, 'overdue'),

    # 연체 후 반납 완료 (2일 연체했다가 반납)
    (7, 3, (today - timedelta(days=20)).isoformat(),
           (today - timedelta(days=10)).isoformat(),
           (today - timedelta(days=8)).isoformat(), 'returned'),

    # VIP 정상 대여중
    (10, 4, (today - timedelta(days=5)).isoformat(),
            (today + timedelta(days=25)).isoformat(), None, 'active'),

    # 정상 반납 완료
    (13, 5, (today - timedelta(days=15)).isoformat(),
            (today - timedelta(days=1)).isoformat(),
            (today - timedelta(days=2)).isoformat(), 'returned'),

    # 연체중 (3일 연체)
    (16, 1, (today - timedelta(days=17)).isoformat(),
            (today - timedelta(days=3)).isoformat(), None, 'overdue'),
]
c.executemany("""INSERT INTO rentals (copy_id, member_id, rental_date, due_date, return_date, status)
              VALUES (?, ?, ?, ?, ?, ?)""", rentals_data)

# 10. 예약 정보 (현재 날짜 기준)
reservations_data = [
    (1, 3, (today - timedelta(days=5)).isoformat(), 'waiting',
     (today + timedelta(days=9)).isoformat()),
    (5, 1, (today - timedelta(days=3)).isoformat(), 'waiting',
     (today + timedelta(days=11)).isoformat())
]
c.executemany("""INSERT INTO reservations (book_id, member_id, reservation_date, status, expiry_date)
              VALUES (?, ?, ?, ?, ?)""", reservations_data)

# 11. 리뷰 데이터 (현재 날짜 기준 최근 리뷰들)
reviews_data = [
    (2, 2, 5, '정말 감동적인 작품이었습니다. 인생 책으로 추천합니다!',
     (today - timedelta(days=10)).isoformat()),
    (3, 5, 4, 'SF의 새로운 가능성을 보여준 소설. 추천해요.',
     (today - timedelta(days=5)).isoformat()),
    (1, 1, 5, '어린 시절 읽었던 기억이 새롭습니다. 명작!',
     (today - timedelta(days=15)).isoformat()),
    (5, 3, 5, '판타지의 정석! 웅장한 세계관에 빠져들었습니다.',
     (today - timedelta(days=12)).isoformat()),
    (6, 4, 5, '디스토피아 소설의 걸작. 현대 사회를 되돌아보게 만드네요.',
     (today - timedelta(days=8)).isoformat()),
    (4, 2, 4, '자아 성찰에 대한 깊은 통찰이 담긴 책. 청소년기에 꼭 읽어야 할 소설입니다.',
     (today - timedelta(days=7)).isoformat()),
    (8, 1, 4, '1920년대 미국 사회의 화려함과 공허함을 잘 표현했어요.',
     (today - timedelta(days=11)).isoformat()),
    (9, 5, 5, '재일교포의 역사를 통해 인간의 존엄성을 느꼈습니다. 강추!',
     (today - timedelta(days=3)).isoformat()),
    (7, 3, 3, '동화지만 철학적 의미가 깊네요. 어른이 읽어도 좋습니다.',
     (today - timedelta(days=9)).isoformat()),
    (10, 4, 4, '10대의 방황과 고민이 생생하게 그려진 작품. 공감 100%',
     (today - timedelta(days=6)).isoformat()),

    # 별로인 리뷰 (1-2점)
    (11, 1, 2, '너무 암울하고 우울해요. 기대했는데 실망스럽습니다.',
     (today - timedelta(days=4)).isoformat()),
    (6, 3, 1, '지루하고 난해합니다. 끝까지 읽기 힘들었어요.',
     (today - timedelta(days=2)).isoformat()),
    (4, 5, 2, '추상적인 내용이 많아서 이해하기 어렵네요. 별로 공감이 안 갑니다.',
     (today - timedelta(days=1)).isoformat()),
    (1, 4, 1, '과대평가된 책 같아요. 스토리가 예상 가능하고 유치합니다.',
     today.isoformat())
]
c.executemany("""INSERT INTO reviews (book_id, member_id, rating, comment, review_date)
              VALUES (?, ?, ?, ?, ?)""", reviews_data)

conn.commit()
print("\n데이터 삽입 완료!")

# === 연체료 자동 계산 ===
print("\n=== 연체료 계산 중... ===\n")

def calculate_late_fees(cursor):
    """연체료 자동 계산 및 삽입"""

    # 연체중인 대여 건 조회 (날짜만 비교)
    cursor.execute("""
        SELECT
            r.rental_id,
            DATE(r.due_date) as due_date,
            DATE(r.return_date) as return_date,
            mt.late_fee_per_day,
            m.name
        FROM rentals r
        JOIN members m ON r.member_id = m.member_id
        JOIN membership_types mt ON m.membership_type_id = mt.type_id
        WHERE (r.return_date IS NULL AND DATE(r.due_date) < DATE('now'))
           OR (r.return_date IS NOT NULL AND DATE(r.return_date) > DATE(r.due_date))
    """)

    for rental_id, due_date, return_date, fee_per_day, member_name in cursor.fetchall():
        # 연체 일수 계산 (SQLite의 날짜 함수 사용하여 일관성 유지)
        cursor.execute("""
            SELECT CAST((JULIANDAY(?) - JULIANDAY(?)) AS INTEGER)
        """, (return_date if return_date else datetime.now().date().isoformat(), due_date))

        overdue_days = cursor.fetchone()[0]

        if overdue_days > 0:
            amount = overdue_days * fee_per_day
            paid_status = 1 if return_date else 0  # 반납했으면 납부완료

            # 이미 연체료가 있는지 확인
            cursor.execute("SELECT fee_id FROM late_fees WHERE rental_id = ?", (rental_id,))
            if not cursor.fetchone():
                payment_date = return_date if paid_status else None
                cursor.execute("""
                    INSERT INTO late_fees (rental_id, amount, calculation_date, paid, payment_date)
                    VALUES (?, ?, DATE('now'), ?, ?)
                """, (rental_id, amount, paid_status, payment_date))
                status = "납부완료" if paid_status else "미납"
                print(f"   {member_name}: {overdue_days}일 연체, {amount:,.0f}원 ({status})")

calculate_late_fees(c)
conn.commit()

# === 복잡한 쿼리 예제 ===
print("\n=== 쿼리 예제 ===\n")

# 1. 책별 평균 평점과 리뷰 수
print("1. 책별 평균 평점:")
c.execute("""
    SELECT b.title, AVG(r.rating) as avg_rating, COUNT(r.review_id) as review_count
    FROM books b
    LEFT JOIN reviews r ON b.book_id = r.book_id
    GROUP BY b.book_id
    HAVING review_count > 0
    ORDER BY avg_rating DESC
""")
for row in c.fetchall():
    print(f"   {row[0]}: {row[1]:.1f}점 ({row[2]}개 리뷰)")

# 2. 회원별 대여 현황 및 연체 정보
print("\n2. 회원별 대여 현황:")
c.execute("""
    SELECT m.name, mt.type_name,
           COUNT(CASE WHEN r.status IN ('active', 'overdue') THEN 1 END) as active_rentals,
           mt.max_books as max_allowed,
           SUM(CASE WHEN lf.paid = 0 THEN lf.amount ELSE 0 END) as unpaid_fees
    FROM members m
    JOIN membership_types mt ON m.membership_type_id = mt.type_id
    LEFT JOIN rentals r ON m.member_id = r.member_id
    LEFT JOIN late_fees lf ON r.rental_id = lf.rental_id
    GROUP BY m.member_id
""")
for row in c.fetchall():
    print(f"   {row[0]} ({row[1]}): {row[2]}/{row[3]}권 대여중, 미납 연체료: {row[4] if row[4] else 0:,.0f}원")

# 3. 가장 인기있는 책 (대여 횟수 기준)
print("\n3. 인기 도서 TOP 5:")
c.execute("""
    SELECT b.title, a.name as author, COUNT(r.rental_id) as rental_count
    FROM books b
    JOIN book_authors ba ON b.book_id = ba.book_id
    JOIN authors a ON ba.author_id = a.author_id
    JOIN book_copies bc ON b.book_id = bc.book_id
    LEFT JOIN rentals r ON bc.copy_id = r.copy_id
    GROUP BY b.book_id
    ORDER BY rental_count DESC
    LIMIT 5
""")
for idx, row in enumerate(c.fetchall(), 1):
    print(f"   {idx}. {row[0]} - {row[1]} ({row[2]}회)")

# 4. 현재 대여 가능한 책 목록
print("\n4. 현재 대여 가능한 책:")
c.execute("""
    SELECT DISTINCT b.title, a.name as author, COUNT(bc.copy_id) as available_copies
    FROM books b
    JOIN book_authors ba ON b.book_id = ba.book_id
    JOIN authors a ON ba.author_id = a.author_id
    JOIN book_copies bc ON b.book_id = bc.book_id
    WHERE bc.status = 'available'
    GROUP BY b.book_id
    ORDER BY available_copies DESC
""")
for row in c.fetchall():
    print(f"   {row[0]} - {row[1]} ({row[2]}권 가능)")

# 5. 현재 연체중인 도서 상세
print("\n5. 현재 연체중인 도서:")
c.execute("""
    SELECT
        m.name,
        b.title,
        DATE(r.due_date) as due_date,
        CAST((JULIANDAY(DATE('now')) - JULIANDAY(DATE(r.due_date))) AS INTEGER) as overdue_days,
        CAST((JULIANDAY(DATE('now')) - JULIANDAY(DATE(r.due_date))) AS INTEGER) * mt.late_fee_per_day as fee
    FROM rentals r
    JOIN members m ON r.member_id = m.member_id
    JOIN membership_types mt ON m.membership_type_id = mt.type_id
    JOIN book_copies bc ON r.copy_id = bc.copy_id
    JOIN books b ON bc.book_id = b.book_id
    WHERE r.return_date IS NULL AND DATE(r.due_date) < DATE('now')
    ORDER BY overdue_days DESC
""")
overdue_list = c.fetchall()
if overdue_list:
    for row in overdue_list:
        print(f"   {row[0]}: {row[1]} (연체 {row[3]}일, 연체료 {row[4]:,}원)")
else:
    print("   연체중인 도서가 없습니다.")

conn.close()
print("\n데이터베이스 연결 종료!")
print(f"\n✅ 모든 날짜가 {today} 기준으로 설정되었습니다.")

테이블 생성 완료!

데이터 삽입 완료!

=== 연체료 계산 중... ===

   이하늘: 5일 연체, 250원 (미납)
   박서준: 2일 연체, 200원 (납부완료)
   김민준: 3일 연체, 300원 (미납)

=== 쿼리 예제 ===

1. 책별 평균 평점:
   노인과 바다: 5.0점 (1개 리뷰)
   반지의 제왕: 5.0점 (1개 리뷰)
   파친코: 5.0점 (1개 리뷰)
   지구 끝의 온실: 4.0점 (1개 리뷰)
   위대한 개츠비: 4.0점 (1개 리뷰)
   The Catcher in the Rye: 4.0점 (1개 리뷰)
   해리 포터와 마법사의 돌: 3.0점 (2개 리뷰)
   데미안: 3.0점 (2개 리뷰)
   1984: 3.0점 (2개 리뷰)
   이상한 나라의 앨리스: 3.0점 (1개 리뷰)
   Brave New World: 2.0점 (1개 리뷰)

2. 회원별 대여 현황:
   김민준 (일반): 2/3권 대여중, 미납 연체료: 300원
   이하늘 (우수): 1/5권 대여중, 미납 연체료: 250원
   박서준 (일반): 0/3권 대여중, 미납 연체료: 0원
   최유리 (VIP): 1/10권 대여중, 미납 연체료: 0원
   정다은 (우수): 0/5권 대여중, 미납 연체료: 0원

3. 인기 도서 TOP 5:
   1. 해리 포터와 마법사의 돌 - J.K. 롤링 (1회)
   2. 노인과 바다 - 어니스트 헤밍웨이 (1회)
   3. 지구 끝의 온실 - 김초엽 (1회)
   4. 데미안 - 헤르만 헤세 (1회)
   5. 반지의 제왕 - J.R.R. 톨킨 (1회)

4. 현재 대여 가능한 책:
   Brave New World - Aldous Huxley (2권 가능)
   파친코 - 이민진 (2권 가능)
   이상한 나라의 앨리스 - 루이스 캐럴 (2권 가능)
   반지의 제왕 - J.R.R. 톨킨 (2권 가능)
   지구 끝의 온실 - 김초엽 (2권 가능)
   해리 포터와 마법사의 돌 - J.K. 롤링 (2권 

## 2. `create_sql_query_chain `

이제, LangChain을 이용해 SQL DB를 처리하는 방법에 대해 알아보겠습니다.

langchain의 부가기능 중 SQLDatabase 를 이용해 db를 불러옵니다.

In [ ]:
from langchain.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///library.db")

db의 `get_table_info()` 결과를 프롬프트에 전달하여 분석을 수행합니다.

In [ ]:
import tiktoken
encoder = tiktoken.encoding_for_model('gpt-4o')
len(encoder.encode(db.get_table_info()))

1830

In [ ]:
print(db.get_table_info())


CREATE TABLE authors (
	author_id INTEGER, 
	name TEXT NOT NULL, 
	birth_year INTEGER, 
	nationality TEXT, 
	PRIMARY KEY (author_id)
)

/*
3 rows from authors table:
author_id	name	birth_year	nationality
1	J.K. 롤링	1965	영국
2	어니스트 헤밍웨이	1899	미국
3	김초엽	1993	대한민국
*/


CREATE TABLE book_authors (
	book_id INTEGER, 
	author_id INTEGER, 
	author_order INTEGER, 
	PRIMARY KEY (book_id, author_id), 
	FOREIGN KEY(book_id) REFERENCES books (book_id), 
	FOREIGN KEY(author_id) REFERENCES authors (author_id)
)

/*
3 rows from book_authors table:
book_id	author_id	author_order
1	1	1
2	2	1
3	3	1
*/


CREATE TABLE book_copies (
	copy_id INTEGER, 
	book_id INTEGER, 
	barcode TEXT, 
	condition TEXT, 
	location TEXT, 
	acquisition_date TEXT, 
	status TEXT DEFAULT 'available', 
	PRIMARY KEY (copy_id), 
	FOREIGN KEY(book_id) REFERENCES books (book_id), 
	UNIQUE (barcode), 
	CHECK (condition IN ('excellent', 'good', 'fair', 'poor')), 
	CHECK (status IN ('available', 'borrowed', 'lost', 'repair'))
)

/*
3 rows 

`create_sql_query_chain`은 질문에 대한 SQL 쿼리를 생성합니다.

In [ ]:
from langchain.chains import create_sql_query_chain

원리 이해를 위해, Built-in Chain을 조금 단순화하여 변환해 보겠습니다.    
(원본 코드: https://api.python.langchain.com/en/latest/_modules/langchain/chains/sql_database/query.html#create_sql_query_chain)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel

sql_prompt = PromptTemplate(template = '''SQL 테이블의 정보와 질문이 주어집니다.
테이블을 참고하여, 질문을 쿼리로 변환하세요.
만약 유저가 데이터의 개수를 지정하지 않는다면, {top_k} 개만 출력하세요.
SQLite의 경우 LIMIT clause 를 사용하면 됩니다.
데이터의 결과를 더 잘 전달하기 위해 정렬할 수 있습니다.
주어진 예시 row에 없는 내용이라도, 쿼리를 작성하세요.

정답과 무관한 컬럼에 대한 쿼리를 수행하지 말고, 필요한 내용만 추출하세요.
아래에 주어지는 컬럼 이름과 테이블 정보를 꼭 참고하세요.
존재하지 않는 컬럼에 대한 쿼리를 하지 않도록 주의하세요.
만약 '오늘'과 같이 현재 날짜에 대한 질문이 들어오면 date() 등을 사용하세요.
컬럼 이름은 "로 구분하고, 전체 출력은 마크다운으로 수행하세요.

테이블 정보: {table_info}
질문: {input}
---
SQL Query:''')


def _create_sql_query_chain(llm, db, prompt, k = 5):

    sql_chain = (
        prompt.partial(table_info = db.get_table_info(), top_k = str(k))
        # 입력 변수 table_info, top_k 미리 추가
        | llm

        | StrOutputParser()
        | (lambda x: x.strip())
        # 공백제거
    )
    return sql_chain

In [ ]:
query_chain = _create_sql_query_chain(llm,
                                     db,
                                     prompt = sql_prompt)

In [ ]:
response = query_chain.invoke("해리 포터와 마법사의 돌을 지금 빌릴 수 있나요?")

response

'```sql\nSELECT "bc"."copy_id", "bc"."barcode", "bc"."status", "bc"."location", "bc"."acquisition_date"\nFROM "books" AS "b"\nJOIN "book_copies" AS "bc" ON "b"."book_id" = "bc"."book_id"\nWHERE "b"."title" = \'해리 포터와 마법사의 돌\'\nORDER BY CASE WHEN "bc"."status" = \'available\' THEN 0 ELSE 1 END, "bc"."copy_id"\nLIMIT 5;\n```'

In [ ]:
# response의 마크다운 형식 처리하기

if "```" in response:
    response = response.split('```sql\n')[1].split('\n```')[0]
print(response)

SELECT "bc"."copy_id", "bc"."barcode", "bc"."status", "bc"."location", "bc"."acquisition_date"
FROM "books" AS "b"
JOIN "book_copies" AS "bc" ON "b"."book_id" = "bc"."book_id"
WHERE "b"."title" = '해리 포터와 마법사의 돌'
ORDER BY CASE WHEN "bc"."status" = 'available' THEN 0 ELSE 1 END, "bc"."copy_id"
LIMIT 5;


제대로 작동하는지 확인해 보겠습니다.

In [ ]:
db.run(response)

"[(2, 'BC00102', 'available', 'A1-1', '2025-05-09'), (3, 'BC00103', 'available', 'A1-1', '2025-05-14'), (1, 'BC00101', 'borrowed', 'A1-1', '2025-05-04')]"

## 쿼리 실행하기

`QuerySQLDatabaseTool`은 쿼리를 받아 실행합니다.

In [ ]:
from langchain_community.tools import QuerySQLDatabaseTool

execute_query = QuerySQLDatabaseTool(db = db)

execute_query.invoke(response)

"[(2, 'BC00102', 'available', 'A1-1', '2025-05-09'), (3, 'BC00103', 'available', 'A1-1', '2025-05-14'), (1, 'BC00101', 'borrowed', 'A1-1', '2025-05-04')]"

두 체인을 연결해 보겠습니다. 중간 파싱 함수가 필요합니다.

In [ ]:
# ```sql, ``` 사이의 문자열만 파싱
def parse_sql(response):
    if "```" in response:
        response = response.split('```sql\n')[1].split('\n```')[0]
    return response

chain = query_chain | parse_sql | execute_query
chain.invoke("해리 포터와 마법사의 돌을 지금 빌릴 수 있나요?")

"[(2, 'BC00102', 'available'), (3, 'BC00103', 'available')]"

임의의 SQL 쿼리를 실행해, 값을 삭제하거나 추가할 수도 있습니다.

In [ ]:
chain.invoke("회원은 누가 있나요?")

"[(1, '김민준'), (2, '이하늘'), (3, '박서준'), (4, '최유리'), (5, '정다은')]"

In [ ]:
chain.invoke("박서준 회원 삭제해줘")

'Error: (sqlite3.ProgrammingError) You can only execute one statement at a time.\n[SQL: -- 박서준 회원과 관련된 데이터 삭제 순서\nDELETE FROM "late_fees"\nWHERE "rental_id" IN (\n  SELECT "rental_id" FROM "rentals"\n  WHERE "member_id" = (\n    SELECT "member_id" FROM "members" WHERE "name" = \'박서준\' LIMIT 1\n  )\n);\n\nDELETE FROM "reviews"\nWHERE "member_id" = (SELECT "member_id" FROM "members" WHERE "name" = \'박서준\' LIMIT 1);\n\nDELETE FROM "reservations"\nWHERE "member_id" = (SELECT "member_id" FROM "members" WHERE "name" = \'박서준\' LIMIT 1);\n\nDELETE FROM "rentals"\nWHERE "member_id" = (SELECT "member_id" FROM "members" WHERE "name" = \'박서준\' LIMIT 1);\n\nDELETE FROM "members"\nWHERE "name" = \'박서준\';]\n(Background on this error at: https://sqlalche.me/e/20/f405)'

In [ ]:
chain.invoke("회원은 누가 있나요?")

"[(1, '김민준'), (2, '이하늘'), (3, '박서준'), (4, '최유리'), (5, '정다은')]"

<br><br>
## 쿼리 생성 후 응답하기
질문이 들어오면, 쿼리를 받아 응답합니다.

In [ ]:
answer_prompt = PromptTemplate(template = """
당신은 매우 활발하고 유머러스한 도서관 사서 AI입니다.
모든 대화는 책 속 유명한 인물의 말투를 그대로 과장되게 따라하세요.
맨 뒤에, 괄호를 통해 누구인지 알려주세요. (OOO 톤으로)

질문과 SQL 쿼리, 쿼리의 실행 결과가 주어집니다.
해당 정보를 바탕으로 질문에 대한 답변을 생성하세요.
만약 결과가 없는 경우, 질문의 내용을 고려하여 적절한 답변을 생성하세요.
대출 조회 결과가 없는 경우, 대출이 가능하다고 알리세요.
보안 규정상 조회가 어려운 답변은, 답변을 거절하세요.

Question: {input}
SQL Query: {query}
SQL Result: {result}
""")

answer_chain = answer_prompt | llm | StrOutputParser()

chain = (
    RunnableParallel(
        input = RunnablePassthrough()).assign(
            query = query_chain | parse_sql).assign(
                result = execute_query).assign(
                    answer= answer_chain)
)

chain.invoke("현재 연체 중인 도서와 연체료가 얼마인지 알려주세요.")

{'input': '현재 연체 중인 도서와 연체료가 얼마인지 알려주세요.',
 'query': 'SELECT b."title", COALESCE(lf."amount", 0) AS "late_fee"\nFROM "rentals" r\nJOIN "book_copies" bc ON r."copy_id" = bc."copy_id"\nJOIN "books" b ON bc."book_id" = b."book_id"\nLEFT JOIN "late_fees" lf ON r."rental_id" = lf."rental_id"\nWHERE r."status" = \'overdue\'\nORDER BY r."due_date" ASC\nLIMIT 5;',
 'result': "[('노인과 바다', 250.0), ('이상한 나라의 앨리스', 300.0)]",
 'answer': '아하, 단서가 여기 있구나! 먼지 쌓인 장부를 들춰보니 현재 연체된 책과 연체료는 다음과 같다 — 꼼꼼히 적어 두게나!\n\n- 노인과 바다: 250원  \n- 이상한 나라의 앨리스: 300원\n\n총 연체료 합계는 550원이다. (정확한 계산은 내 관찰력 덕분이지.) 조속히 반납하라, 그렇지 않으면 다음 단서는 더 불쾌할 테다! (셜록 홈즈 톤으로)'}

테스트 질문
- 이메일이 haneul@example.com인 회원의 정보를 알려주세요.
- 해리 포터와 마법사의 돌을 지금 빌릴 수 있나요?
- 김민준 회원이 지금까지 빌린 책 목록을 보여주세요.
- 현재 연체 중인 도서와 연체료가 얼마인지 알려주세요.
- 판타지 장르에서 가장 인기 있는 책 3개를 추천해주세요.
- 최근 6개월간 연체 이력이 없는 회원을 찾아서 우수 회원으로 업그레이드 추천해주세요.

## Human(LLM) In the Loop 만들기

코드는 잘 구성되어 있지만, 위에서 진행했던 것처럼 위험한 쿼리를 실행하는 것은 사전에 방지할 필요가 있습니다.    

복잡한 로직도 가능하지만, 여기서는 중간 쿼리를 검증하여 Y/N으로 판독할 수 있는 단순한 모듈을 추가해 보겠습니다.    

   
질문과 쿼리를 보고, 해당 내용이 안전하지 않으면 이후 과정을 실행하지 않겠습니다.

In [ ]:
validation_prompt = PromptTemplate(template=
    """아래의 SQL 쿼리를 서버에서 실행하고자 합니다.
쿼리의 안전성을 평가하세요.
Approve하면 Y, 아니면 아무 키나 입력하세요.

---

Question = {input}

---
SQL Query = {query}
""")
# Prompt 지만 LLM이 아닌 인간에게 전달

def human_approval(msg):
    val_msg = validation_prompt.format(query = msg['query'], input = msg['input'])

    print(type(msg))
    resp = input(val_msg)

    if resp.upper() == 'Y':
        return execute_query

    else:
        return '보안 규정상 실행할 수 없음'



In [ ]:
# Parallel 대신 Passthrough 사용하는 것도 가능
# 이 경우 입력에 input이 전달되어야 함

sql_chain_with_validation = RunnablePassthrough.assign(query =
                                                       query_chain | parse_sql).assign(
                                                           result = human_approval).assign(
                                                               answer = answer_chain)

sql_chain_with_validation.invoke({"input": "현재 연체 중인 도서와 연체료가 얼마인지 알려주세요."})

<class 'dict'>
아래의 SQL 쿼리를 서버에서 실행하고자 합니다.
쿼리의 안전성을 평가하세요.
Approve하면 Y, 아니면 아무 키나 입력하세요.

---

Question = 현재 연체 중인 도서와 연체료가 얼마인지 알려주세요.

---
SQL Query = SELECT
  b."title",
  r."rental_id",
  COALESCE(SUM(lf."amount"), 0) AS "late_fee_due"
FROM "rentals" r
JOIN "book_copies" bc ON r."copy_id" = bc."copy_id"
JOIN "books" b ON bc."book_id" = b."book_id"
LEFT JOIN "late_fees" lf ON lf."rental_id" = r."rental_id" AND lf."paid" = 0
WHERE r."status" = 'overdue'
GROUP BY r."rental_id", b."title"
ORDER BY "late_fee_due" DESC
LIMIT 5;
y


{'input': '현재 연체 중인 도서와 연체료가 얼마인지 알려주세요.',
 'query': 'SELECT\n  b."title",\n  r."rental_id",\n  COALESCE(SUM(lf."amount"), 0) AS "late_fee_due"\nFROM "rentals" r\nJOIN "book_copies" bc ON r."copy_id" = bc."copy_id"\nJOIN "books" b ON bc."book_id" = b."book_id"\nLEFT JOIN "late_fees" lf ON lf."rental_id" = r."rental_id" AND lf."paid" = 0\nWHERE r."status" = \'overdue\'\nGROUP BY r."rental_id", b."title"\nORDER BY "late_fee_due" DESC\nLIMIT 5;',
 'result': "[('이상한 나라의 앨리스', 6, 300.0), ('노인과 바다', 2, 250.0)]",
 'answer': '흠흠! 내 증거(데이터)를 들추어 보니, 연체 중인 책은 두 권이로군. 사건은 단순하다: 연체된 책과 각 연체료는 다음과 같다!\n\n- "이상한 나라의 앨리스" — 연체료: 300.0원 (대출ID: 6)  \n- "노인과 바다" — 연체료: 250.0원 (대출ID: 2)\n\n총 연체료 합계는 550.0원이다. 빠르게 해결하면 도서들이 다시 평화롭게 책장으로 돌아갈 것이오. 서가의 평온을 위해 속히 납부하시길 권하노라! (셜록 홈즈 톤으로)'}

# [실습] LLM으로 validation 검증 체인 추가하기   

위의 `sql_chain_with_validation` 구조를 수정하여,    
Human Approval 대신 LLM이 이를 검증하는 체인을 추가하세요.   
다음은 예시 프롬프트입니다.

```python   
아래의 SQL 쿼리를 서버에서 실행하고자 합니다.
SQL 쿼리는 새로운 데이터를 추가하거나, 기존의 값을 변경하거나 삭제해서는 안 됩니다.
쿼리의 안전성에 대해 먼저 30자 이내로 설명하세요.
안전하면 "분류 결과: Y", 아니면 "분류 결과: N"을 출력하세요.

SQL Query = {query}
```

In [ ]:
llm_validation_prompt = PromptTemplate(template = """
아래의 SQL 쿼리를 서버에서 실행하고자 합니다.
SQL 쿼리는 새로운 데이터를 추가하거나, 기존의 값을 변경하거나 삭제해서는 안 됩니다.
쿼리의 안전성에 대해 먼저 30자 이내로 아래의 포맷에 맞게 설명하세요.
포맷)
안전하면 "분류 결과: Y, 근거: ...이기 때문에 안전합니다.", 아니면 "분류 결과: N, 근거: ...이기 때문에 안전하지 않습니다."을 출력하세요.

SQL Query = {query}
""")

def llm_approval(msg):
    chain = llm_validation_prompt | llm | StrOutputParser()
    result = chain.invoke(msg)
    print(result)

    if "분류 결과: Y" in result:
       return human_approval(msg)


    else:
        return '보안 규정상 실행할 수 없음'


In [ ]:
# Parallel 대신 Passthrough 사용하는 것도 가능
# 이 경우 입력에 input이 전달되어야 함

sql_chain_with_validation = RunnablePassthrough.assign(query =
                                                       query_chain | parse_sql).assign(
                                                           result = llm_approval).assign(
                                                               answer = answer_chain)

sql_chain_with_validation.invoke({"input": "연체 도서에 1권을 추가해주세요."})
# sql_chain_with_validation.invoke({"input": "현재 연체 중인 도서와 연체료가 얼마인지 알려주세요."})

분류 결과: N, 근거: 추가이기 때문에 안전하지 않습니다.


{'input': '연체 도서에 1권을 추가해주세요.',
 'query': 'INSERT INTO "rentals" ("rental_id", "copy_id", "member_id", "rental_date", "due_date", "return_date", "status")\nVALUES (4, 3, 2, \'2025-09-30\', \'2025-10-15\', NULL, \'overdue\');',
 'result': '보안 규정상 실행할 수 없음',
 'answer': '아하! 흥미로운 미스터리로군요 — 연체 도서 한 권을 추가해달라구요. 그러나, 증거가 명확합니다: 실제 DB 변경은 나의 손으로는 할 수 없소. 보안 규정상 실행을 거절하겠나이다. (내가 범죄를 저지를 리가 없지 않겠습니까?) (셜록 홈즈 톤으로)\n\n대신, 내가 탐정처럼 단계별로 명확히 알려주겠소 — 주어진 INSERT 문을 그대로 사용하거나 안전하게 실행하는 방법과 검증 절차를 아래에 적어두었소.\n\n1) 실행 불가 안내 (거절)\n- 보안 규정상 직접 데이터베이스에 쓰기(INSERT/UPDATE/DELETE)를 수행할 수 없습니다. 따라서 실행은 불가합니다.\n\n2) 제공된 INSERT 문 (수정이나 확인 후 DB에서 실행)\n- INSERT INTO "rentals" ("rental_id", "copy_id", "member_id", "rental_date", "due_date", "return_date", "status")\n  VALUES (4, 3, 2, \'2025-09-30\', \'2025-10-15\', NULL, \'overdue\');\n\n3) 실행 전에 확인할 것들 (꼭 검증하세요)\n- rental_id(4)가 이미 존재하지 않는지 확인: SELECT 1 FROM rentals WHERE rental_id = 4;\n- copy_id(3)와 member_id(2)가 각각 존재하는지 확인: SELECT 1 FROM copies WHERE copy_id = 